# Machine learning in bioinformatics

Dataset: a set of DNA sequences labeled with 0 and 1

Aim: build a classifier that will predict the label for new DNA sequences

ML workflow:

- import the dataset
- prepare ML method(s) and encoding(s) and the set of hyperparameters to be evaluated
- set up the evaluation of methods and encodings
- select the final model

## Importing the data

The data is located in data.fa file. For each sequence, there is a line starting with `>` with the sequence label (0 or 1), and a new line with the DNA sequence itself:
```
> 1
AGCGAGGCAGGTGCGGTCACGTGACCCGGCGGCGCTGCGGGGCAGCGGCCATTTTGCGGGGCGGCC
> 0
AGCGAGGGCGCTCGGAGTGCGACGTTTTGGCACCAGGCGGGGCGCACGGCATTGCCAAGCGGCCGC
```

Function load_data will import this data into a pandas dataframe.

In [3]:
from util import load_data

dataset = load_data("data.fa")

dataset.head(10)

,sequence,label
0,AGCGAGGCAGGTGCGGTCACGTGACCCGGCGGCGCTGCGGGGCAGC...,1
1,AGCGAGGGCGCTCGGAGTGCGACGTTTTGGCACCAGGCGGGGCGCA...,0
2,TCTGGTCCCACTTAGCGGGAGAAACTCGGGCCACGTGACTGTCTGA...,1
3,TCGCGTCTCCTGCCTCCCTGTCCCGGGGCGAGGACTGTCTCAGAAA...,0
4,ACTCCCCAGGTCAGCTGACGCTGCGTCCTCAGCGGGCTGTCAAGTG...,1
5,AGAAGAACTCGGGGCTGGAACCTGTCGTACGGGTGTCCAGCAGCTG...,0
6,GCTGGGAGGCGGTCACGTGGCGCAGGATGCACCCAACACCACCGTC...,1
7,GCCGCAGTCGGGCCCAGTGGATGGCGCACTCGACGGGAACCCGCGT...,0
8,TGAAGCCGCCATTTCCCCGGCCCAGCCACCACGTGACCCTTCTGCG...,1
9,TCCCGGCGAGCACGGACATACCCCATAGCTTTCTCCTCCGCCCGTG...,0


## Defining ML methods and encodings

In this section, a few ML methods will be defined. You can use any other suitable ML method from scikit-learn library that is already installed in this environment.

One encoding is already defined and can be used right away. It represents a letter in a sequence as a vector of length 4 (4 possible letters: A, C, G, T) with one 1 and 0s elsewhere to indicate which letter is present at the given position. To represent the full sequence, these vectors are just concatenated one after another. For example, for sequence `AG` this representation looks like this:

```
[1, 0, 0, 0, 0, 0, 1, 0]
```

The first four digits describe letter A, the second four - letter G.

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from util import encode_onehot


# ML methods - defining ML methods to be used

logistic_regression = LogisticRegression(penalty='l1', C=10, solver='saga', max_iter=500)
random_forest = RandomForestClassifier()

# encodings - encodes the data right away and returns a dictionary with encoded_data, labels, and feature_names

one_hot_encoded_data = encode_onehot(dataset)

print(one_hot_encoded_data['encoded_data'])

[[1. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


### Defining a custom encoding

Another encoding that can be used in this setting is based on the frequency of subsequences within a sequence.

For example, if the original sequence is `AGCGAG`, subsequences of length 3 (3-mers) are:
- `AGC`
- `GCG`
- `CGA`
- `GAG`.

The encoding would then represent each sequence by the frequency of each possible 3-mer. For sequence `AGCGAG` it would be:

```
[0., 0., 0., 0., 0., 0., 0., 0., 0, 0.25, 0.,
 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
 0., 0., 0.25, 0., 0., 0., 0., 0., 0., 0., 0.,
 0., 0.25, 0., 0., 0., 0.25, 0., 0., 0., 0., 0.,
 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
 0., 0., 0., 0., 0., 0., 0., 0., 0.]
```

The first number is then the frequency of 3-mer `AAA`, the second the frequency of `AAC`, third `AAG`, ..., and the last one is the frequency of `TTT`.

Here you can implement the function that will encode the dataset by representing each DNA sequence by the frequency of its subsequences.

The function should take a pandas DataFrame as input with columns `sequence` and `label` (as shown above). It should output a dictionary with the following keys: `encoded_data` (matrix where each row represents one sequence and each column represents the frequency of one k-mer), `labels` (an array of labels for each sequence), and `feature_names` (a list of features names, e.g., `AAA`, `AAC`, `ACA`, etc.)

In [5]:
import pandas as pd
from util import ALPHABET

# all letters (nucleotides) that can be used in a sequence:
print(ALPHABET)

# define a custom encoding:

def encode_kmer_frequencies(dataset: pd.DataFrame, k: int) -> dict:
    
    # add your code here and fill in the returned object
    
    return {
        'encoded_data': None,
        'labels': None,
        'feature_names': None
    }

['A', 'C', 'G', 'T']


In [4]:
import numpy as np
from util import test_kmer_encoding

test_kmer_encoding(encode_kmer_frequencies_func=encode_kmer_frequencies)
    

k-mer encoding works!


# Selecting optimal ML model and encoding

To find the best ML model and encoding, we can use the defined models and encodings, train and assess their performance and select the best one using the procedure called cross-validation.

In [13]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import copy
from util import standardize

encoded_datasets = [one_hot_encoded_data] # if implemented k-mer encoding, add k-mer encoded dataset to this list
ml_models = [logistic_regression, random_forest] # add here all ML models to assess

metric_func = accuracy_score
n_splits = 3
performances = {}

kf = KFold(n_splits=n_splits)

for split_index, (train_indices, test_indices) in enumerate(kf.split(range(dataset.shape[0]))):
    
    print(f"Assessing split: {split_index+1}/{n_splits}")

    for i, encoded_dataset in enumerate(encoded_datasets):
        
        train_data = encoded_dataset['encoded_data'][train_indices]
        train_labels = encoded_dataset['labels'][train_indices]
        
        test_data = encoded_dataset['encoded_data'][test_indices]
        test_labels = encoded_dataset['labels'][test_indices]
        
        train_data, test_data = standardize(train_data, test_data)
        
        for j, ml_model in enumerate(ml_models):
            
            combination = f"encoding_{i+1}_ml_{j+1}"
            print(f"Assessing combination: {combination}")
            
            ml_model = copy.deepcopy(ml_model)
            ml_model.fit(X=train_data, y=train_labels)
            
            performance = metric_func(y_pred=ml_model.predict(test_data), y_true=test_labels)
                        
            if combination in performances:
                performances[combination].append(performance)
            else:
                performances[combination] = [performance]
                
print(performances)

optimal_combination = max(performances, key=lambda k: sum(performances[k])/n_splits)
        
print(f"\nOptimal combination is {optimal_combination}.")

Assessing split: 1/3
Assessing combination: encoding_1_ml_1
Assessing combination: encoding_1_ml_2
Assessing split: 2/3
Assessing combination: encoding_1_ml_1
Assessing combination: encoding_1_ml_2
Assessing split: 3/3
Assessing combination: encoding_1_ml_1
Assessing combination: encoding_1_ml_2
{'encoding_1_ml_1': [0.6086232728483436, 0.5897286499084402, 0.5703346096221076], 'encoding_1_ml_2': [0.5684201764607958, 0.5575994672881638, 0.5448643249542201]}

Optimal combination is encoding_1_ml_1.


## Refit optimal model to the full dataset

Running the following cell will retrain the model to the full dataset and make it ready to apply on the new data.

In [14]:
optimal_method = copy.deepcopy(ml_models[int(optimal_combination.split("_ml_")[1])-1])
optimally_encoded_data = copy.deepcopy(encoded_datasets[int(optimal_combination.split("_")[1])-1])

optimal_method.fit(optimally_encoded_data['encoded_data'], optimally_encoded_data['labels'])

LogisticRegression(C=10, max_iter=500, penalty='l1', solver='saga')

# Test the optimal combination on a new dataset

Import the new dataset, encode it, test the performance of the optimal model on the new data, and report the new score.

In [7]:
test_dataset = load_data("test_data.fa")

encoded_test_dataset = encode_onehot(test_dataset) 
# alternatively: encode_kmer_frequencies(test_dataset, k=k), depending what was chosen as the optimal encoding in the previous step

test_predictions = optimal_method.predict(encoded_test_dataset)

performance = metric_func(y_true=test_dataset['labels'], y_pred=test_predictions)
print(f"Performance on test dataset: {performance}")

FileNotFoundError: [Errno 2] No such file or directory: 'test_data.fa'